In [1]:
import os
import re
import numpy as np
import rasterio
import imageio
import matplotlib.pyplot as plt
from rasterio.enums import Resampling
from matplotlib import colors
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO

In [2]:

def generate_channel_evolution_gif_with_dynamic_plot(
    directory,
    output_path="channel_evolution_with_plot.gif",
    duration=500,
    font_size=24
):
    """
    Generates a GIF showing river channel evolution with a dynamically updating AR plot.
    """
    pattern = re.compile(r'.*?_(\d{4})_DSWE.*\.tif$', re.IGNORECASE)
    file_info = []

    for file in os.listdir(directory):
        match = pattern.match(file)
        if match:
            year = int(match.group(1))
            file_info.append((year, os.path.join(directory, file)))

    if not file_info:
        raise ValueError(f"No matching .tif files found in directory: {directory}")

    file_info.sort()
    rasters = []

    for _, path in file_info:
        with rasterio.open(path) as src:
            data = src.read(1, resampling=Resampling.nearest)
            data = (data > 0).astype(np.uint8)
            rasters.append(data)

    raster_stack = np.stack(rasters)
    Aw = np.mean([np.count_nonzero(r == 1) for r in raster_stack])

    frames = []
    cumulative_wet = np.zeros_like(rasters[0], dtype=np.uint8)
    AR_series = []
    years = [y for y, _ in file_info]

    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        font = ImageFont.load_default()
        print("⚠️  TrueType font not found. Falling back to default font.")

    for idx, (year, raster) in enumerate(zip(years, rasters)):
        status = raster.copy()
        status[(cumulative_wet == 1) & (raster == 0)] = 2
        cumulative_wet = np.maximum(cumulative_wet, raster)

        AR = (np.count_nonzero(status == 2) + np.count_nonzero(status == 1) - Aw)
        AR_percent = (AR / Aw) * 100 if Aw > 0 else 0
        AR_series.append(AR)

        # Create annotated raster image
        rgb = np.zeros((*status.shape, 3), dtype=np.uint8)
        rgb[status == 0] = [245, 245, 220]
        rgb[status == 1] = [0, 0, 255]
        rgb[status == 2] = [0, 128, 0]
        img = Image.fromarray(rgb)
        draw = ImageDraw.Draw(img)

        label = (
            f"Year: {year}  |  "
            f"Aw: {int(Aw):,} px  |  "
            f"AR: {int(AR):,} px  |  "
            f"AR/Aw: {AR_percent:.1f}%"
        )

        bbox = draw.textbbox((0, 0), label, font=font)
        text_width = bbox[2] - bbox[0]
        img_width, _ = img.size
        position = ((img_width - text_width) // 2, 10)
        draw.text(position, label, fill="black", font=font)

        # Create dynamic AR plot
        fig, ax = plt.subplots(figsize=(img_width / 100, 2.5), dpi=100)
        ax.plot(years[:idx + 1], AR_series, marker='o', color='black')
        ax.set_title("Reworked Area (AR) Over Time")
        ax.set_xlabel("Year")
        ax.set_ylabel("AR (px)")
        ax.grid(True)
        plt.tight_layout()

        buf = BytesIO()
        plt.savefig(buf, format='png')
        plt.close(fig)
        buf.seek(0)

        plot_img = Image.open(buf)
        plot_img = plot_img.resize((img_width, plot_img.height))

        # Stack the raster + plot vertically
        total_height = img.height + plot_img.height
        combined = Image.new("RGB", (img_width, total_height), (255, 255, 255))
        combined.paste(img, (0, 0))
        combined.paste(plot_img, (0, img.height))

        frames.append(combined)

    frames[0].save(output_path, save_all=True, append_images=frames[1:], duration=duration, loop=0)
    return output_path

In [11]:
generate_channel_evolution_gif_with_dynamic_plot(
    directory=r"C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Minjiang\reach_10\Cleaned",
    output_path=r"C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Minjiang\GIFs\Minjiang_reach_10_fromCleaned.gif",
    duration=500,
    font_size=36
)



'C:\\Users\\huckr\\Desktop\\UCSB\\Dissertation\\Data\\RiverMapping\\RiverMasks\\Minjiang\\GIFs\\Minjiang_reach_10_fromCleaned.gif'